# AUXILIARY FUNCTIONS

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from string import upper
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage


# FUNCTIONS IA636

def iah2percentile(h, p):
    s = h.sum()
    k = ((s-1) * p/100.)+1
    dw = np.floor(k)
    up = np.ceil(k)
    hc = np.cumsum(h)
    if isinstance(p, int):
        k1 = np.argmax(hc>=dw)
        k2 = np.argmax(hc>=up)
    else:
        k1 = np.argmax(hc>=dw[:, np.newaxis], axis=1)
        k2 = np.argmax(hc>=up[:, np.newaxis], axis=1)
    d0 = k1 * (up-k)
    d1 = k2 * (k-dw)
    
    return np.where(dw==up, k1, d0+d1)

def iaconv(f, h):
    f, h = np.asarray(f), np.asarray(h,float)
    if len(f.shape) == 1: f = f[newaxis,:]
    if len(h.shape) == 1: h = h[newaxis,:]
    if f.size < h.size:
        f, h = h, f
    g = zeros(array(f.shape) + array(h.shape) - 1)
    if f.ndim == 2:
        H,W = f.shape
        for (r,c) in transpose(nonzero(h)):
            g[r:r+H, c:c+W] += f * h[r,c]

    if f.ndim == 3:
        D,H,W = f.shape
        for (d,r,c) in transpose(nonzero(h)):
            g[d:d+D, r:r+H, c:c+W] += f * h[d,r,c]

    return g


def ianormalize(f, range=[0,255]):
        
    f = np.asarray(f)
    range = np.asarray(range)
    if f.dtype.char in ['D', 'F']:
        raise Exception, 'error: cannot normalize complex data'
    faux = ravel(f).astype(float)
    minimum = faux.min()
    maximum = faux.max()
    lower = range[0]
    upper = range[1]
    if upper == lower:
        g = ones(f.shape) * maximum
    if minimum == maximum:
        g = ones(f.shape) * (upper + lower) / 2.
    else:
        g = (faux-minimum)*(upper-lower) / (maximum-minimum) + lower
    g = reshape(g, f.shape)
        
    if f.dtype == uint8:
        if upper > 255: 
            raise Exception,'ianormalize: warning, upper valuer larger than 255. Cannot fit in uint8 image'
    g = g.astype(f.dtype) # set data type of result the same as the input image
    return g

    
# FUNCTIONS FROM IA870

def iaadd4dil(f, c):

    if not c:
        return f
    if f.dtype == 'float64':
        y = f + c
    else:
        y = np.asarray(f,int64) + c
        k1,k2 = ialimits(f)
        y = ((f==k1) * k1) + ((f!=k1) * y)
        y = clip(y,k1,k2)
    a = y.astype(f.dtype)
    return a

def iabinary(f, k1=1):
    f = np.asarray(f)
    y = f >= k1            
    return y

def iadil(f, b=None):

    if b is None: b = iasecross()
        
    if len(f.shape) == 1: f = f[newaxis,:]
    h,w = f.shape
    x,v = iamat2set(b)
    if len(x)==0:
        y = (ones((h,w)) * ialimits(f)[0]).astype(f.dtype)
    else:
        if iaisbinary(v):
            v = iaintersec( iagray(v,'int32'),0)
        mh,mw = max(abs(x)[:,0]),max(abs(x)[:,1])
        y = (ones((h+2*mh,w+2*mw)) * ialimits(f)[0]).astype(f.dtype)
        for i in range(x.shape[0]):
            if v[i] > -2147483647:
                y[mh+x[i,0]:mh+x[i,0]+h, mw+x[i,1]:mw+x[i,1]+w] = maximum(
                    y[mh+x[i,0]:mh+x[i,0]+h, mw+x[i,1]:mw+x[i,1]+w], iaadd4dil(f,v[i]))
        y = y[mh:mh+h, mw:mw+w]
            
    return y

def iagray(f, TYPE="uint8", k1=None):
    ff = array([0],TYPE)
    kk1,kk2 = ialimits(ff)
    if k1!=None:
        kk2=k1
    if   TYPE == 'uint8'  : y = where(f,kk2,kk1).astype(uint8)
    elif TYPE == 'uint16' : y = where(f,kk2,kk1).astype(uint16)
    elif TYPE == 'int32'  : y = where(f,kk2,kk1).astype(int32)
    elif TYPE == 'int64'  : y = where(f,kk2,kk1).astype(int64)
    elif TYPE == 'float64': y = where(f,kk2,kk1).astype(float64)
    else:
        assert 0, 'type not supported:'+TYPE
    return y

def ianeg(f):

    if ialimits(f)[0] == (- ialimits(f)[1]):
        y = -f
    else:
        y = ialimits(f)[0] + ialimits(f)[1] - f
    y = y.astype(f.dtype)     
    return y

def iaero(f, b=None):

    if b is None: b = iasecross()
    y = ianeg( iadil( ianeg(f),iasereflect(b)))
    return y

def iaisbinary(f):
        
    return f.dtype == bool

def ialimits(f):
    code = f.dtype
    if   code == bool:   y=array([0,1],'bool')
    elif code == uint8:  y=array([0,255],'uint8')
    elif code == uint16: y=array([0,(2**16)-1],'uint16')
    elif code == int32:  y=array([-((2**31)-1),(2**31)-1],'int32')
    elif code == int64:  y=array([-((2**63)-1), (2**63)-1],'int64')
    elif code == float64:y=array([-Inf,Inf],'float64')
    else:
        assert 0,'ialimits: Does not accept this typecode:%s' % code
    return y

def iamat2set(A):

    if len(A.shape) == 1: A = A[newaxis,:]
    offsets = nonzero(ravel(A) - ialimits(A)[0])
    if type(offsets) == type(()):
        offsets = offsets[0]        # for compatibility with numarray
    if len(offsets) == 0: return ([],[])
    (h,w) = A.shape
    x = range(2)
    x[0] = offsets/w - (h-1)/2
    x[1] = offsets%w - (w-1)/2
    x = transpose(x)
    CV = x,ravel(A)[offsets]           
    return CV

def iaintersec(f1, f2, *args):
        
    y = minimum(f1,f2)
    for f in args:
        y = minimum(y,f)
    return y.astype(f1.dtype)

def iase2off(Bc,option='neigh'):

    h,w = Bc.shape
    hc,wc = h/2,w/2
    B = Bc.copy()
    B[hc,wc] = 0  # remove origin
    off = transpose(B.nonzero()) - array([hc,wc])
    if option == 'neigh':
        return off  # 2 columns x n. of neighbors rows
    elif option == 'fw':
        i = off[:,0] * w + off[:,1] 
        return off[i>0,:]  # only neighbors higher than origin in raster order
    elif option == 'bw':
        i = off[:,0] * w + off[:,1] 
        return off[i<0,:]  # only neighbors less than origin in raster order
    else:
        assert 0,'options are neigh, fw or bw. It was %s'% option
        return None
  

def iasedil(B1, B2):        
    assert (iaisbinary(B1) or (B1.dtype == int32) or (B1.dtype == int64) or (B1.dtype == float64)) and \
               (iaisbinary(B2) or (B2.dtype == int32) or (B2.dtype == int64) or (B2.dtype == float64)), \
               'iasedil: s.e. must be binary, int32, int64 or float64'
    if len(B1.shape) == 1: B1 = B1[newaxis,:]
    if len(B2.shape) == 1: B2 = B2[newaxis,:]
    if B1.dtype=='bool' and B2.dtype == 'bool':
        Bo = iabinary([0])
    else:
        Bo = array(ialimits(B1)[0]).reshape(1)
        if iaisbinary(B1):
            Bo = array(ialimits(B2)[0]).reshape(1)
            B1 = iagray(B1,B2.dtype,0)
        if iaisbinary(B2):
            Bo = array(ialimits(B1)[0]).reshape(1)
            B2 = iagray(B2,B1.dtype,0)
    x,v = iamat2set(B2)
    if len(x):
        for i in range(x.shape[0]):
            s = iaadd4dil(B1,v[i])
            st= iasetrans(s,x[i])
            Bo = iaseunion(Bo,st)
    return Bo

def iasesum(B, N=1): 
    if N==0:
        if iaisbinary(B): return iabinary([1])
        else:             return array([0],int32) # identity
    NB = B
    for i in range(N-1):
        NB = iasedil(NB,B)
    return NB    
    
def iaseunion(B1, B2):

    if B1.dtype != B2.dtype:
        print ('B1=',B1)
        print ('B2=',B2)
    assert B1.dtype == B2.dtype, \
        'iaseunion: Cannot have different datatypes: \
        %s and %s' % (str(B1.dtype), str(B2.dtype))
    type1 = B1.dtype
    #if len(B1) == 0: return B2
    if len(B1.shape) == 1: B1 = B1[newaxis,:]
    if len(B2.shape) == 1: B2 = B2[newaxis,:]
    if B1.shape <> B2.shape:
        inf = ialimits(B1)[0]
        h1,w1 = B1.shape
        h2,w2 = B2.shape
        H,W = max(h1,h2),max(w1,w2)
        Hc,Wc = (H-1)/2,(W-1)/2    # center
        BB1,BB2 = np.asarray(B1),np.asarray(B2)
        B1, B2  = inf * ones((H,W)), inf *ones((H,W))
        dh1s , dh1e = (h1-1)/2 , (h1-1)/2 + (h1+1)%2 # deal with even and odd dimensions
        dw1s , dw1e = (w1-1)/2 , (w1-1)/2 + (w1+1)%2
        dh2s , dh2e = (h2-1)/2 , (h2-1)/2 + (h2+1)%2
        dw2s , dw2e = (w2-1)/2 , (w2-1)/2 + (w2+1)%2
        B1[ Hc-dh1s : Hc+dh1e+1  ,  Wc-dw1s : Wc+dw1e+1 ] = BB1
        B2[ Hc-dh2s : Hc+dh2e+1  ,  Wc-dw2s : Wc+dw2e+1 ] = BB2
    B = maximum(B1,B2).astype(type1)
    return B

def iasetrans(Bi, t):
  
    x,v=iamat2set(Bi)
    Bo = iaset2mat((x+t,v))
    Bo = Bo.astype(Bi.dtype)
                    
    return Bo

def iasecross(r=1):
    B = iasesum( iabinary([[0,1,0],
                           [1,1,1],
                           [0,1,0]]),r)      
    return B


def iaserot(B, theta=45, DIRECTION="CLOCKWISE"):
    
    DIRECTION = upper(DIRECTION)            
    if DIRECTION == "ANTI-CLOCKWISE":
        theta = -theta
    SA = iamat2set(B)
    theta = pi * theta/180
    (y,v)=SA
    if len(y)==0: return iabinary([0])
    x0 = y[:,1] * cos(theta) - y[:,0] * sin(theta)
    x1 = y[:,1] * sin(theta) + y[:,0] * cos(theta)
    x0 = int32((x0 +0.5)*(x0>=0) + (x0-0.5)*(x0<0))
    x1 = int32((x1 +0.5)*(x1>=0) + (x1-0.5)*(x1<0))
    x = transpose(array([transpose(x1),transpose(x0)]))
    BROT = iaset2mat((x,v))   
    return BROT


def iasereflect(Bi):
    Bo = iaserot(Bi, 180)
    return Bo

def iaNlut(s,offset):

    H,W = s
    n = H*W
    hi = arange(H).reshape(-1,1)
    wi = arange(W).reshape(1,-1) 
    hoff = offset[:,0]
    woff = offset[:,1]
    h = hi + hoff.reshape(-1,1,1)
    w = wi + woff.reshape(-1,1,1)
    h[(h<0) | (h>=H)] = n
    w[(w<0) | (w>=W)] = n
    Nlut = clip(h * W + w,0,n)
    return Nlut.reshape(offset.shape[0],-1).transpose()

 
def iasebox(r=1):
    B = iasesum( iabinary([[1,1,1],
                           [1,1,1],
                           [1,1,1]]),r)
    return B

def iasedisk(r=3, DIM="2D", METRIC="EUCLIDEAN", FLAT="FLAT", h=0):

    METRIC = upper(METRIC)
    FLAT   = upper(FLAT)            
    assert DIM=='2D','Supports only 2D structuring elements'
    if FLAT=='FLAT': y = iabinary([1])
    else:            y = int32([h])
    if r==0: return y
    if METRIC == 'EUCLIDEAN':
        v = arange(-r,r+1)
        x = resize(v, (len(v), len(v)))
        y = transpose(x)
        Be = iabinary(sqrt(x*x + y*y) <= (r+0.5))
        if FLAT=='FLAT':
            return Be
        be = h + int32( sqrt( maximum((r+0.5)*(r+0.5) - (x*x) - (y*y),0)))
        be = iaintersec( iagray(Be,'int32'),be)
        return be
    else:
        assert 0,'Non valid metric'
                    
                
    return B

def ialabelflat(f, Bc=iasecross(), delta=0):
    r = f.astype(np.int) + 1
    return ialabelflat_unionfind(fr,Bc,delta)
 
# implementation by union find
def Find(x,parents): # uses path compression
    if parents[x] == x:
        return x
    else:
        parents[x] = Find(parents[x],parents)
    return parents[x]

def Union(n, p,parents):
    r = Find(n,parents)
    if r != p:
        parents[r] = p

def ialabelflat_unionfind(img,Bc,delta):
    imshape = img.shape
    imsize  = img.size

    # Offsets and neighbors
    offsets = iase2off(Bc,'fw')
    neighbors = iaNlut(imshape,offsets)
    parents = np.arange(imsize,dtype = int)
    # Raster image and no-nzero pixels
    img = img.ravel()
    nonzero_nodes = np.nonzero(img)[0]
    img =  np.concatenate((img,np.array([0])))
    g = np.zeros(imsize, dtype = int)
    cur_label = 0
    # pass 1: backward scan, forward neighbors
    for p in nonzero_nodes[::-1]:
        v = img[p]
        for nb in neighbors[p]:
            if img[nb] and (abs(v - img[nb]) <= delta):
                Union(nb,p,parents)
    # pass 2_1: root labeled
    pbool = parents[nonzero_nodes] == nonzero_nodes
    labels = np.arange(1,pbool.sum()+1)
    g[nonzero_nodes[pbool]] = labels
    # pass 2_2: labeling root descendants
    for p in nonzero_nodes[~pbool]:
        g[p] = g[parents[p]]
    return g.reshape(imshape)